<a href="https://colab.research.google.com/github/cds21199/research2021/blob/main/GalaxyMorphologyClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Galaxy Morphology Classification - Chloë Smith
## University of the Witwatersrand
## Supervisor - Dr. Ritesh Ajoodha

## Setup and imports

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
#!pip install imgaug

In [ ]:
#!pip3 install keras-visualizer

In [ ]:
pwd

In [ ]:
cd drive/My Drive/Research Project/galaxy-zoo-kaggle/training/images

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import random

import imgaug as ia
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, Input

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model

In [ ]:
cd ..

# Load dataframe containing image filenames and probabilities for each question (our targets)

In [ ]:
# load training labels
df_dist = pd.read_csv("solutions/training_solutions_rev1.csv")
df_dist.head()

In [ ]:
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1',
           'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3',
           'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
           'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6',
           'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2',
           'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4', 'Class11.5', 'Class11.6']
#df_dist['Max'] = df_dist[classes].idxmax(axis=1)
#df_dist[['GalaxyID', 'Max']]

### Convert IDs to filenames (i.e. add extension .jpg)

In [ ]:
df_dist['GalaxyID'] = df_dist['GalaxyID'].astype(str).apply(lambda x: str(x) + ".jpg")
df_dist

# Load images and resize, no augmentation yet

In [ ]:
def preprocessing(img):
  out = img
  # TODO

  assert np.shape(out) == np.shape(img)
  return out

In [ ]:
# ImageDataGenerator does our preprocessing
datagen = ImageDataGenerator()
# some resizing is done here, to 224x224 images
train_generator = datagen.flow_from_dataframe(dataframe=train, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)
valid_generator = datagen.flow_from_dataframe(dataframe=test, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)

# Benchmark transfer learning model (ResNet)

We are exploring the effects of different feature sets, this model uses no handcrafted feature extraction.

We train ResNet-50 on the images that have only been scaled down.

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer to train for our specific dataset
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# add a prediction layer (softmax) for our 37 target features
predictions = Dense(37, activation='softmax')(x)

# train only top layers
# freeze the hidden layers from ResNet50
for layer in base_model.layers:
  layer.trainable = False

# model to train
model = Model(inputs=base_model.input, outputs=predictions)

# compile model, done after freezing layers
model.compile(optimizer='Adam', loss='mse', metrics=["mae", "acc"])

In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model
#plot_model(model, to_file="../model-11-17.png")

In [ ]:
# train the modecl
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
with tf.device('/gpu:0'):
  model.fit(train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            validation_data=valid_generator,
            validation_steps=STEP_SIZE_VALID,
            epochs=3)

# save the model
model.save('../ResNet50/11-17')

## load saved model and run for more epochs

In [ ]:
# load the model
#new_model = keras.models.load_model('../custom/11-07')
#np.testing.assert_allclose(model.predict(train_generator),
                #new_model.predict(train_generator),
                #1e-5)
#new_model.evaluate(train_generator)

In [ ]:
# fit the model
#checkpoint = keras.callbacks.ModelCheckpoint('../checkpoints/11-31', monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]
#STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#history = new_model.fit(train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            validation_data=valid_generator,
            validation_steps=STEP_SIZE_VALID,
            epochs=3)

In [ ]:
# save new model
#new_model.save('../ResNet50/11-31')

# New model (benchmark?)

In [ ]:
model = Sequential()
# input 224 x 224 x 3
model.add(Conv2D(224, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# added for VGG-16
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# end

model.add(Flatten())
#model1.add(Dense(2048)) # changed from 512 to 4096 to 2048
#model1.add(Activation('relu'))
model.add(Dense(1024)) # added for VGG-16, halved all nodes
model.add(Activation('relu'))
model.add(Dense(512)) # added, reduced for a bottleneck
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(37, activation='softmax'))

In [ ]:
print(model.summary())

### Model with no augmentation

In [ ]:
# split train into train and dev sets
train, test = train_test_split(df_dist, test_size=0.2, random_state=42, shuffle=True)
#display(train)

# load images from buffer, dataset is large
# original images are 424 x 424 pixels
datagen = ImageDataGenerator()
# some resizing is done here, to 224x224 images
train_generator = datagen.flow_from_dataframe(dataframe=train, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)
valid_generator = datagen.flow_from_dataframe(dataframe=test, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)

In [ ]:
#plot_model(model, to_file="../model-11-18.png")

In [ ]:
# train the model
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(optimizer=opt, loss='mse', metrics=["acc", tf.keras.metrics.RootMeanSquaredError()])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history = model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=50)

# save the model
model.save('../custom/11-19')

### In-place data augmentation

In [ ]:
# split train into train and dev sets
train, test = train_test_split(df_dist, test_size=0.2, random_state=42, shuffle=True)
#display(train)

# load images from buffer, dataset is large
# original images are 424 x 424 pixels
datagen = ImageDataGenerator(width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=45,
                             fill_mode='nearest')
# some resizing is done here, to 224x224 images
train_generator = datagen.flow_from_dataframe(dataframe=train, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)
valid_generator = datagen.flow_from_dataframe(dataframe=test, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)

In [ ]:
# train the model
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError(), metrics=["acc", tf.keras.metrics.RootMeanSquaredError()])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history = model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=30)

# save the model
model.save('../custom/11-23')

In [ ]:
dir(valid_generator)

In [ ]:
out = model.predict(x_test)

In [ ]:
m = tf.keras.metrics.CosineSimilarity(axis=1)
acc = m.update_state(out, y_test)
acc = m.result().numpy()
acc

### Model, loss functions and learning rate scheduler

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10 or lr <= 0.000001:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [ ]:
def rmse(y_true, y_pred):
  # root mean squared error
  return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true)))

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model1 = Sequential()
# input 224 x 224 x 3
model1.add(Conv2D(224, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

# added for VGG-16
model1.add(Conv2D(256, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(256, (3, 3)))
model1.add(Activation('relu'))
model1.add(Conv2D(256, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(256, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(256, (3, 3)))
model1.add(Activation('relu'))
model1.add(Conv2D(256, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))
# end

model1.add(Flatten())
#model1.add(Dense(2048)) # changed from 512 to 4096 to 2048
#model1.add(Activation('relu'))
model1.add(Dense(1024)) # added for VGG-16, halved all nodes
model1.add(Activation('relu'))
model1.add(Dense(512)) # added, reduced for a bottleneck
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(37, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model1.compile(optimizer=opt, loss=rmse, metrics=["acc"])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history = model1.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=50,
          callbacks=[callback])

In [ ]:
# save new model
model1.save('../aug/11-17')

## Feature extraction and data augmentation

In [ ]:
def augment_set(data, seq):
  aug = []
  for image in data:
    seq_det = seq.to_deterministic()
    img = seq_det.augment_image(image)
  return aug

In [ ]:
# TODO augment training set
seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Crop(percent=(0, 0.1)), # random crops
      iaa.LinearContrast((0.75, 1.5)),
      #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
      #iaa.Multiply((0.8, 1.2), per_channel=0.2),
      iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        mode='edge'
    )
])

aug = []
for image in train:
  seq_det = seq.to_deterministic()
  img = seq_det.augment_image(image.image)
  mask = seq_det.augment_image(image.mask)
  mask[mask < 0.5] = 0
  mask[mask >= 0.5] = 1
  aug.append(LabelledImage(image.num, preprocess(img), preprocess(mask)))

print(len(aug))

In [ ]:
# split train into train and dev sets
train, test = train_test_split(df_dist, test_size=0.2, random_state=42, shuffle=True)
#display(train)

# load images from buffer, dataset is large
# original images are 424 x 424 pixels
datagen = ImageDataGenerator(width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=45,
                             fill_mode='nearest')
# some resizing is done here, to 45x45 images
train_generator = datagen.flow_from_dataframe(dataframe=train, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)
valid_generator = datagen.flow_from_dataframe(dataframe=test, 
                                      directory="images/images_training_rev1", 
                                      x_col="GalaxyID", 
                                      y_col=classes, 
                                      class_mode="raw", 
                                      target_size=(224,224), 
                                      batch_size=64)

# Testing

## code blocks

In [ ]:
model = Sequential()
# input 224 x 224 x 3
model.add(Conv2D(224, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# added for VGG-16
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# end

model.add(Flatten())
#model1.add(Dense(2048)) # changed from 512 to 4096 to 2048
#model1.add(Activation('relu'))
model.add(Dense(1024)) # added for VGG-16, halved all nodes
model.add(Activation('relu'))
model.add(Dense(512)) # added, reduced for a bottleneck
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(37, activation='softmax'))
print(model.summary())

In [ ]:
#plot_model(model, to_file="../model-11-17.png")

In [ ]:
#!pip3 install keras-visualizer

In [ ]:
from keras_visualizer import visualizer
visualizer(model, format='png', view=True)